<a href="https://colab.research.google.com/github/Russianbluecat/Funnel_analysis_example/blob/main/251027_Pyhton_user_behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Table description**

*   User Table - 가입자 ID / 가입일자 / 생년월일
*   Ordersheet Table - 주문 ID / 가입자 ID / 주문일자 / 주문금액
*   Event Table - Device ID / Event Name / Event_timestamp


## Data source :
구글 스프레드 시트 링크는 코드에 포함되어 있음

## Task_1

1-1
* 전체 회원에 대해 연령 그룹별 회원 카운트,
* 주문 경험이 있는 회원 수
* 주문 경험 회원수 비율
* 구매 금액의 합/구매회원 수
* 연령 그룹 - 20세 미만 / 20-24세, 25-29세, 30-34세, 35-39세, 40-44세, 45세 이상

1-2
* 특정 연도이후 구매 월 별, new order 여부별 order count , avg_order_amount

1-3
* 특정 연도이후 구매 월별 Order Count, Order Amount, agg Order Count, agg Order Amount
* 예시 : 2025년 2월//
 2월 의 Order Count, Order Amount 및 1월 ~
2월 Order Count 누적 합계 / 1월~2월 Order Amount 합계


### Library setting

In [ ]:
!pip install --upgrade gspread gspread_dataframe google-auth

In [1]:
#
import gspread
from google.colab import auth
auth.authenticate_user()


In [2]:
#
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
import pandas as pd
import numpy as np


In [3]:
# Colab 기본 credentials 로 인증
import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

In [ ]:
#구글 스프레드 시트 URL은 오른쪽 상단 공유 -> 링크 복사 누르면 된다.

# user_df
user_url = "https://docs.google.com/spreadsheets/d/1DsGUvy9oqxuEelH1PdO3RmRFOgOHLWnJ_7JKTVsasnk/edit?usp=sharing"
user_ws = gc.open_by_url(user_url).sheet1
user_df = get_as_dataframe(user_ws, evaluate_formulas=True, dtype=str)

print("User DataFrame:")
print(user_df.head())
print("\n")

# order_df
order_url = "https://docs.google.com/spreadsheets/d/1HKlck-JAQYn0nhRlGJtRf7dwgY1svaabWPUpXrulL-0/edit?usp=sharing"
order_ws = gc.open_by_url(order_url).sheet1
order_df = get_as_dataframe(order_ws, evaluate_formulas=True, dtype=str)

print("\nOrder DataFrame:")
order_df.head()



User DataFrame:
  id            joined  birth_date
0  1   2020-01-01 0:00  1970-01-01
1  2  2020-01-19 10:40  1970-06-19
2  3  2020-02-06 21:20  1970-12-06
3  4   2020-02-25 8:00  1971-05-25
4  5  2020-03-14 18:40  1971-11-10



Order DataFrame:


,ordersheet_id,user_id,order_timestamp,order_amount
0,1,52,2021-01-01,129101
1,2,93,2021-01-08,13748
2,3,15,2021-01-15,23545
3,4,72,2021-01-22,137659
4,5,61,2021-01-29,83530


### 만나이 구하기 + 연령 그룹 구하기
 User table에서 생년월일 컬럼을 이용해서 만나이를 구하고 연령 그룹 구할 것

 그 이후 order sheet와 join 할 것  

In [ ]:
print(user_df['birth_date'].dtype)

object


In [ ]:
#오늘 날짜 구하기
today = pd.to_datetime("today")

#birth_date 컬럼을 datetime 형태로 변환
user_df['birth_date'] = pd.to_datetime(user_df['birth_date'], errors='coerce')

#연도 빼고
# 생일 월이 더 크면 나이 1세 감소
# 생일 월이 같고 생일 일이 더 크면 나이 1세 감소
user_df['age'] = (
    today.year - user_df['birth_date'].dt.year
    - (
        (today.month < user_df['birth_date'].dt.month)
        | (
            (today.month == user_df['birth_date'].dt.month)
            & (today.day < user_df['birth_date'].dt.day)
        )
    )
)
print(user_df.shape)
user_df.head(5)


(100, 4)


,id,joined,birth_date,age
0,1,2020-01-01 0:00,1970-01-01,55
1,2,2020-01-19 10:40,1970-06-19,55
2,3,2020-02-06 21:20,1970-12-06,54
3,4,2020-02-25 8:00,1971-05-25,54
4,5,2020-03-14 18:40,1971-11-10,54


In [ ]:
bins = [-1, 19, 24, 29, 34, 39, 44, np.inf]
labels = ['A.19 below', 'B.20-24', 'C.25-29', 'D.30-34', 'E.35-39', 'F.40-44', 'G.45+']
user_df['age_band'] = pd.cut(user_df['age'], bins=bins, labels=labels)

user_df['age_band'].value_counts(sort=False).reindex(labels)

,count
age_band,
A.19 below,22
B.20-24,11
C.25-29,11
D.30-34,11
E.35-39,10
F.40-44,11
G.45+,24


In [ ]:
print(user_df['id'].dtype)

object


### ARPPU

*   전체 회원에 대해 연령 그룹별 회원 카운트,
*   주문 경험이 있는 회원 수
*   주문 경험 회원수 비율
*   구매 금액의 합/구매회원 수


In [ ]:
# order_df의 데이터 타입 변환
order_df['user_id'] = pd.to_numeric(order_df['user_id'], errors='coerce')
order_df['order_amount'] = pd.to_numeric(order_df['order_amount'], errors='coerce')
user_df['id'] = pd.to_numeric(user_df['id'], errors='coerce')


In [ ]:
# 1. user_df와 order_df 조인
merged_df = user_df.merge(
    order_df,
    left_on='id',
    right_on='user_id',
    how='left'
)

print("Merged DataFrame:")
merged_df.head(5)

Merged DataFrame:


,id,joined,birth_date,age,age_band,ordersheet_id,user_id,order_timestamp,order_amount
0,1,2020-01-01 0:00,1970-01-01,55,G.45+,113,1.0,2023-02-24,144508.0
1,1,2020-01-01 0:00,1970-01-01,55,G.45+,147,1.0,2023-10-20,14014.0
2,1,2020-01-01 0:00,1970-01-01,55,G.45+,193,1.0,2024-09-06,38251.0
3,2,2020-01-19 10:40,1970-06-19,55,G.45+,17,2.0,2021-04-23,122859.0
4,2,2020-01-19 10:40,1970-06-19,55,G.45+,21,2.0,2021-05-21,17392.0


In [ ]:
print(merged_df['order_timestamp'].dtype)

object


In [ ]:

# 2. 연령대별 전체 회원 수
result = merged_df.groupby('age_band', observed=False).agg(
    join_user=('id', 'nunique')  # 전체 회원 카운트 (중복 제거)
).reset_index()

# 3. 연령대별 주문 경험 있는 회원 수 및 ARPPU 계산
order_stats = merged_df[merged_df['order_amount'].notna()].groupby('age_band', observed=False).agg(
    order_user=('id', 'nunique'),           # 주문 경험 있는 회원 수
    total_amount=('order_amount', 'sum')    # 총 구매액
).reset_index()

# 4. ARPPU 계산 (구매 금액의 합 / 구매 회원 수)
order_stats['avg_order_amount'] = (order_stats['total_amount'] / order_stats['order_user']).round(0).astype(int)

# 5. 결과 합치기
result = result.merge(order_stats[['age_band', 'order_user', 'avg_order_amount']], on='age_band', how='left')

# 6. 주문 경험 회원 수 비율 계산
result['order_ratio'] = (result['order_user'] / result['join_user']).round(2)

# 7. NaN 처리 (주문 경험 없는 연령대)
result['order_user'] = result['order_user'].fillna(0).astype(int)
result['avg_order_amount'] = result['avg_order_amount'].fillna(0).astype(int)
result['order_ratio'] = result['order_ratio'].fillna(0)

result.head(30)
#order_stats.head(5)


,age_band,join_user,order_user,avg_order_amount,order_ratio
0,A.19 below,22,21,163716,0.95
1,B.20-24,11,9,196825,0.82
2,C.25-29,11,10,207883,0.91
3,D.30-34,11,9,174524,0.82
4,E.35-39,10,10,176440,1.00
5,F.40-44,11,9,216906,0.82
6,G.45+,24,19,217313,0.79


In [ ]:
# 전체 회원 카운트 (중복 제거)
# 주문 경험 있는 회원 수
# result = merged_df.groupby('age_band', observed=False).agg(
#     join_user=('id', 'nunique'),
#     order_user=('user_id','nunique'),
#     total_amount=('order_amount', 'sum')
# ).reset_index()

# # 새로운 컬럼 추가
# result['order_ratio'] = (result['order_user'] / result['join_user']).round(2)
# result['avg_order_amount'] = (result['total_amount'] / result['order_user']).round(0).astype(int)

# result.head(10)

### 특정 연도이후 count
특정 연도이후 구매 월별 Order Count, Avg Order Amount
특정 연도 이후 월별/ 가입 후 신규 주문여부 별 / Order Count / Avg Order Amount
예: 2024년 1월 - 가입 후 첫 주문 여부 / 주문 건수 / 평균 주문 금액 (사람, 주문 수로 나눈 것)


In [ ]:
# 1. order_amount가 있는 것만 필터링
merged_filtered = merged_df[merged_df['order_amount'].notna()].copy()

# 2. order_timestamp를 datetime으로 변환
merged_filtered['order_timestamp'] = pd.to_datetime(merged_filtered['order_timestamp'])

# 3. user_id별로 order_timestamp 순서대로 번호 매기기
merged_filtered = merged_filtered.sort_values(['user_id', 'order_timestamp'])
merged_filtered['order_seq'] = merged_filtered.groupby('user_id').cumcount() + 1

# 4. is_new_order 컬럼 추가 (1번이면 Y, 2이상이면 N)
merged_filtered['is_new_order'] = np.where(
    merged_filtered['order_seq'] == 1,
    'Y',    # 조건이 True일 때
    'N'     # 조건이 False일 때
)

print(merged_filtered.head(10))

# 5. 2024년 이후만 필터링
merged_2024 = merged_filtered[merged_filtered['order_timestamp'] >= '2024-01-01'].copy()
print("\n===========\n")
print(merged_2024.head(5))

# 6. 월 추출
merged_2024['order_month'] = merged_2024['order_timestamp'].dt.strftime('%Y-%m')

# 7. 월별, is_new_order별 집계
result_2 = merged_2024.groupby(['order_month', 'is_new_order']).agg(
    order_count=('ordersheet_id', 'count'),        # 주문 건수
    order_user=('user_id', 'nunique'),             # 주문 인원 (중복 제외)
    total_amount=('order_amount', 'sum')           # 총 주문 금액
).reset_index()
print("\n===========\n")
print(result.head(5))

# 8. 평균 계산(주문 금액 / 회원수 , 주문금액 / 주문 건수)
result_2['avg_order_amount_user'] = (result_2['total_amount'] / result_2['order_user']).round(0).astype(int)
result_2['avg_order_amount_order'] = (result_2['total_amount'] / result_2['order_count']).round(0).astype(int)

# 9. 필요한 컬럼만 선택
result_2 = result_2[['order_month', 'is_new_order', 'order_count', 'avg_order_amount_user', 'avg_order_amount_order']]





   id            joined birth_date  age age_band ordersheet_id  user_id  \
0   1   2020-01-01 0:00 1970-01-01   55    G.45+           113      1.0   
1   1   2020-01-01 0:00 1970-01-01   55    G.45+           147      1.0   
2   1   2020-01-01 0:00 1970-01-01   55    G.45+           193      1.0   
3   2  2020-01-19 10:40 1970-06-19   55    G.45+            17      2.0   
4   2  2020-01-19 10:40 1970-06-19   55    G.45+            21      2.0   
5   2  2020-01-19 10:40 1970-06-19   55    G.45+            59      2.0   
6   2  2020-01-19 10:40 1970-06-19   55    G.45+            73      2.0   
7   3  2020-02-06 21:20 1970-12-06   54    G.45+            14      3.0   
8   3  2020-02-06 21:20 1970-12-06   54    G.45+            45      3.0   
9   3  2020-02-06 21:20 1970-12-06   54    G.45+           146      3.0   

  order_timestamp  order_amount  order_seq is_new_order  
0      2023-02-24      144508.0          1            Y  
1      2023-10-20       14014.0          2            N  


In [ ]:
result_2.head(10)

,order_month,is_new_order,order_count,avg_order_amount_user,avg_order_amount_order
0,2024-01,N,3,68394,68394
1,2024-01,Y,1,20647,20647
2,2024-02,N,3,96173,96173
3,2024-02,Y,1,12368,12368
4,2024-03,N,3,82874,82874
5,2024-03,Y,2,61064,61064
6,2024-04,N,3,98534,98534
7,2024-04,Y,1,134862,134862
8,2024-05,N,4,80772,80772
9,2024-05,Y,1,32677,32677


In [ ]:
#검증

from google.colab import files

# CSV 파일로 저장
merged_df.to_csv('merged_df.csv', index=False, encoding='utf-8-sig')

# 파일 다운로드
files.download('merged_df.csv')

#검증 완료

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# 엑셀 파일로 저장
merged_df.to_excel('merged_df.xlsx', index=False, engine='openpyxl')

# 파일 다운로드
files.download('merged_df.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 월별 누적 합계 / 누적 주문

In [ ]:
# 1. order_amount가 있는 것만 filtering
agg_merged_filtered = merged_df[merged_df['order_amount'].notna()].copy()

# 2. order_timestamp를 datetime으로 변환
agg_merged_filtered['order_timestamp'] = pd.to_datetime(agg_merged_filtered['order_timestamp'])

# 3. 2024년 이후만 filtering
agg_merged_2024 = agg_merged_filtered[agg_merged_filtered['order_timestamp'] >= '2024-01-01'].copy()

# 4. 주문 월 추출
agg_merged_2024['order_month'] = agg_merged_2024['order_timestamp'].dt.strftime('%Y-%m')

# 5. Monthly Sum / Count
agg_result = agg_merged_2024.groupby('order_month').agg(
    order_count=('ordersheet_id', 'count'),      # 주문 건수
    order_amount=('order_amount', 'sum')         # 주문 금액
).reset_index()

# 6. Cummulative Sum
agg_result['agg_order_count'] = agg_result['order_count'].cumsum()
agg_result['agg_order_amount'] = agg_result['order_amount'].cumsum()

# 7. 금액 정수로 변환
agg_result['order_amount'] = agg_result['order_amount'].round(0).astype(int)
agg_result['agg_order_amount'] = agg_result['agg_order_amount'].round(0).astype(int)

agg_result.head(10)


  order_month  order_count  order_amount  agg_order_count  agg_order_amount
0     2024-01            4        225828                4            225828
1     2024-02            4        300886                8            526714
2     2024-03            5        370751               13            897465
3     2024-04            4        430465               17           1327930
4     2024-05            5        355764               22           1683694
5     2024-06            4        379781               26           2063475
6     2024-07            4        409926               30           2473401
7     2024-08            5        393901               35           2867302
8     2024-09            4        310243               39           3177545
9     2024-10            4        177326               43           3354871


## Task_2 (퍼널 분석 funnel analysis)
Table Description
- Event Table 이 있음
- 컬럼은 Device ID / Event Name / Event_timestamp 임
- Event_Name에는 Pageview(제품상세조회) / AddtoCart / Purchase가 있음

Problem
- pageview_count
- addtocart_5min_count	(제품 상세 조회 후 5분 내 장바구니 담기)
- addtocart_5min_ratio
- purchase_30min_count (제품 상세 조회 후 30분 내 구매)
- purchase_30min_ratio


### 예시데이터 만들기 by Claude


In [5]:
#Claude가 예시 데이터 생성함

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# 시드 고정
random.seed(42) #Always Same Data
np.random.seed(42)

# 파라미터 설정
device_ids = np.arange(1, 101)  # 100명의 유저
products = [f"P{i:03}" for i in range(1, 21)]  # 상품 20개
start_date = datetime(2025, 11, 1, 9, 0, 0)

rows = []

for device in device_ids:
    # 하루 동안 여러 세션 발생 (1~5회)
    for _ in range(random.randint(1, 5)):
        session_start = start_date + timedelta(
            days=random.randint(0, 2),
            hours=random.randint(0, 14),
            minutes=random.randint(0, 59)
        )

        product = random.choice(products)
        current_time = session_start

        # 1. PAGEVIEW는 기본
        rows.append([device, product, 'PAGEVIEW', current_time])

        # 2. 장바구니 담기 (50% 확률, 1~30분 사이 랜덤)
        if random.random() < 0.5:
            current_time += timedelta(minutes=random.randint(1, 30))
            rows.append([device, product, 'ADDTOCART', current_time])

            # 2-1. 장바구니 담은 후 구매 (40% 확률, 1~60분 사이)
            if random.random() < 0.4:
                current_time += timedelta(minutes=random.randint(1, 60))
                rows.append([device, product, 'PURCHASE', current_time])

        # 3. 장바구니 없이 바로 구매 (20% 확률, 1~45분 사이)
        elif random.random() < 0.25:  # 50% 안 담은 사람 중 25% = 전체의 12.5%
            current_time += timedelta(minutes=random.randint(1, 45))
            rows.append([device, product, 'PURCHASE', current_time])

# DataFrame 생성
event_df = pd.DataFrame(rows, columns=['device_id', 'product_id', 'event_name', 'event_timestamp'])
event_df.sort_values(['device_id', 'product_id','event_timestamp'], inplace=True)
event_df.reset_index(drop=True, inplace=True)

# 데이터 확인
print(f"총 이벤트 수: {len(event_df)}")
print(f"PAGEVIEW: {len(event_df[event_df['event_name']=='PAGEVIEW'])}")
print(f"ADDTOCART: {len(event_df[event_df['event_name']=='ADDTOCART'])}")
print(f"PURCHASE: {len(event_df[event_df['event_name']=='PURCHASE'])}")
print("\n샘플 데이터:")
event_df.head(10)

총 이벤트 수: 537
PAGEVIEW: 303
ADDTOCART: 148
PURCHASE: 86

샘플 데이터:


,device_id,product_id,event_name,event_timestamp
0,1,P008,PAGEVIEW,2025-11-01 20:17:00
1,1,P008,ADDTOCART,2025-11-01 20:41:00
2,1,P008,PURCHASE,2025-11-01 21:29:00
3,2,P002,PAGEVIEW,2025-11-01 18:27:00
4,2,P002,ADDTOCART,2025-11-01 18:34:00
5,2,P002,PURCHASE,2025-11-01 19:13:00
6,2,P002,PAGEVIEW,2025-11-02 18:16:00
7,2,P006,PAGEVIEW,2025-11-01 21:51:00
8,2,P011,PAGEVIEW,2025-11-01 12:48:00
9,2,P011,ADDTOCART,2025-11-01 13:01:00


In [ ]:
event_df.sort_values([ 'product_id','device_id','event_timestamp'], inplace=True)
event_df.head(20)

In [ ]:
from google.colab import files

# 엑셀 파일로 저장
event_df.to_excel('event_df.xlsx', index=False, engine='openpyxl')

# 파일 다운로드
files.download('event_df.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
pattern_check = event_df.groupby(['device_id', 'product_id'])['event_name'].apply(
    lambda x: ' → '.join(x)
).value_counts()
# pattern_check.head(30)
# 1. 총 이벤트 수
print(f"00.총 이벤트 수: {len(event_df)}")

# 2. (device_id, product_id) 조합 수
print(f"01.(device, product) 조합 수: {len(event_df.groupby(['device_id', 'product_id']))}")

# 3. pattern_check의 count 합
print(f"02.패턴 카운트 합: {pattern_check.sum()}")

00.총 이벤트 수: 537
01.(device, product) 조합 수: 287
02.패턴 카운트 합: 287


### 전환 Count

In [20]:
# ============================================
# 퍼널 분석: 상품별 전환율 계산
# ============================================

def analyze_funnel(event_df):
    """
    전체 이벤트 데이터를 받아서 Product별 퍼널 분석 결과 반환
    """
    results = []

    # Product별로 순회
    for product in event_df['product_id'].unique():
        product_events = event_df[event_df['product_id'] == product]

        # 1. PAGEVIEW한 고유 디바이스 수
        pageview_devices = product_events[
            product_events['event_name'] == 'PAGEVIEW'
        ]['device_id'].unique()
        pageview_count = len(pageview_devices)

        # 2. 각 디바이스별로 전환 여부 체크
        addtocart_5min_count = 0
        purchase_30min_count = 0

        for device in pageview_devices:
            # 이 디바이스의 이 상품에 대한 모든 이벤트 추출
            device_product_events = product_events[
                product_events['device_id'] == device
            ].sort_values('event_timestamp').reset_index(drop=True)

            # === 여기서 바로 전환 체크 (함수 호출 대신 인라인) ===
            converted_addtocart_5min = False
            converted_purchase_30min = False

            # 모든 PAGEVIEW 순회
            pageviews = device_product_events[device_product_events['event_name'] == 'PAGEVIEW']

            for _, pv_row in pageviews.iterrows():
                pv_time = pv_row['event_timestamp']

                # 이 PAGEVIEW 이후의 이벤트들
                after_events = device_product_events[
                    device_product_events['event_timestamp'] > pv_time
                ]

                # 5분 이내 ADDTOCART 체크
                if not converted_addtocart_5min:
                    next_addtocart = after_events[after_events['event_name'] == 'ADDTOCART']
                    if not next_addtocart.empty:
                        first_ac = next_addtocart.iloc[0]
                        time_diff_min = (first_ac['event_timestamp'] - pv_time).total_seconds() / 60
                        if time_diff_min <= 5:
                            converted_addtocart_5min = True

                # 30분 이내 PURCHASE 체크
                if not converted_purchase_30min:
                    next_purchase = after_events[after_events['event_name'] == 'PURCHASE']
                    if not next_purchase.empty:
                        first_pu = next_purchase.iloc[0]
                        time_diff_min = (first_pu['event_timestamp'] - pv_time).total_seconds() / 60
                        if time_diff_min <= 30:
                            converted_purchase_30min = True

                # 둘 다 찾았으면 조기 종료
                if converted_addtocart_5min and converted_purchase_30min:
                    break

            # === 전환 체크 끝 ===

            # 카운트 증가
            if converted_addtocart_5min:
                addtocart_5min_count += 1
            if converted_purchase_30min:
                purchase_30min_count += 1

        # 3. 비율 계산
        addtocart_5min_ratio = addtocart_5min_count / pageview_count if pageview_count > 0 else 0
        purchase_30min_ratio = purchase_30min_count / pageview_count if pageview_count > 0 else 0

        results.append({
            'product_id': product,
            'pageview_count': pageview_count,
            'addtocart_5min_count': addtocart_5min_count,
            'addtocart_5min_ratio': round(addtocart_5min_ratio, 3),
            'purchase_30min_count': purchase_30min_count,
            'purchase_30min_ratio': round(purchase_30min_ratio, 3)
        })

    # DataFrame으로 변환 및 정렬
    result_df = pd.DataFrame(results)
    result_df = result_df.sort_values('product_id', ascending=True).reset_index(drop=True)

    return result_df

In [21]:

# ============================================
# Step 2: 실행 및 결과 확인
# ============================================

# 분석 실행
funnel_result = analyze_funnel(event_df)

print("=" * 80)
print("제품별 퍼널 분석 결과")
print("=" * 80)
funnel_result.head(20)




제품별 퍼널 분석 결과


,product_id,pageview_count,addtocart_5min_count,addtocart_5min_ratio,purchase_30min_count,purchase_30min_ratio
0,P001,10,1,0.100,2,0.200
1,P002,16,0,0.000,2,0.125
2,P003,17,2,0.118,1,0.059
3,P004,14,1,0.071,3,0.214
4,P005,12,2,0.167,1,0.083
5,P006,13,1,0.077,3,0.231
6,P007,7,1,0.143,3,0.429
7,P008,18,2,0.111,2,0.111
8,P009,14,1,0.071,1,0.071
9,P010,13,1,0.077,1,0.077


In [17]:

# 요약 통계
print("=" * 80)
print("전체 요약")
print("=" * 80)
print(f"00.총 제품 수: {len(funnel_result)}")
print(f"01.평균 5분 이내 전환율: {funnel_result['addtocart_5min_ratio'].mean():.1%}")
print(f"02.평균 30분 이내 구매율: {funnel_result['purchase_30min_ratio'].mean():.1%}")


전체 요약
00.총 제품 수: 20
01.평균 5분 이내 전환율: 9.7%
02.평균 30분 이내 구매율: 15.1%


In [18]:

# 전환율 최고 제품 찾기
best_addtocart = funnel_result.loc[funnel_result['addtocart_5min_ratio'].idxmax()]
best_purchase = funnel_result.loc[funnel_result['purchase_30min_ratio'].idxmax()]

print(f"\n00.5분 이내 전환율 최고 제품: {best_addtocart['product_id']} ({best_addtocart['addtocart_5min_ratio']:.1%})")
print(f"01.30분 이내 구매율 최고 제품: {best_purchase['product_id']} ({best_purchase['purchase_30min_ratio']:.1%})")
print(f"02.PAGEVIEW 최다 제품: {funnel_result.iloc[0]['product_id']} ({funnel_result.iloc[0]['pageview_count']}회)")


00.5분 이내 전환율 최고 제품: P016 (26.7%)
01.30분 이내 구매율 최고 제품: P007 (42.9%)
02.PAGEVIEW 최다 제품: P001 (10회)


### 여기서부터는 참고만

In [ ]:
test_pageviews = pd.DataFrame({
    'device_id': [1, 1, 2],
    'product_id': ['P001', 'P001', 'P002'],
    'event_name': ['PAGEVIEW', 'PAGEVIEW', 'PAGEVIEW'],
    'event_timestamp': ['2025-11-01 17:01:00', '2025-11-01 17:05:00', '2025-11-01 18:00:00']
})

test_pageviews.head(5)


,device_id,product_id,event_name,event_timestamp
0,1,P001,PAGEVIEW,2025-11-01 17:01:00
1,1,P001,PAGEVIEW,2025-11-01 17:05:00
2,2,P002,PAGEVIEW,2025-11-01 18:00:00


In [ ]:
for index, row in test_pageviews.iterrows():
    print(f"Index: {index}")
    print(f"Row: {row}")
    print(f"Timestamp: {row['event_timestamp']}")
    print("-" * 40)

Index: 0
Row: device_id                            1
product_id                        P001
event_name                    PAGEVIEW
event_timestamp    2025-11-01 17:01:00
Name: 0, dtype: object
Timestamp: 2025-11-01 17:01:00
----------------------------------------
Index: 1
Row: device_id                            1
product_id                        P001
event_name                    PAGEVIEW
event_timestamp    2025-11-01 17:05:00
Name: 1, dtype: object
Timestamp: 2025-11-01 17:05:00
----------------------------------------
Index: 2
Row: device_id                            2
product_id                        P002
event_name                    PAGEVIEW
event_timestamp    2025-11-01 18:00:00
Name: 2, dtype: object
Timestamp: 2025-11-01 18:00:00
----------------------------------------


In [ ]:
import pandas as pd
from datetime import timedelta

# 이벤트를 시간 순으로 정렬
event_df = event_df.sort_values(['device_id', 'product_id', 'event_timestamp']).reset_index(drop=True)

# PAGEVIEW, ADDTOCART, PURCHASE 분리
pageview_df = event_df[event_df['event_name'] == 'PAGEVIEW']
addtocart_df = event_df[event_df['event_name'] == 'ADDTOCART']
purchase_df = event_df[event_df['event_name'] == 'PURCHASE']

# ---------------------------
# 🔹 각 PAGEVIEW별로 5분 내 ADDTOCART 여부 / 30분 내 PURCHASE 여부 체크
# ---------------------------

result_rows = []

for idx, row in pageview_df.iterrows():
    device = row['device_id']
    product = row['product_id']
    ts = row['event_timestamp']

    # ① 5분 내 ADDTOCART 존재 여부
    addtocart_flag = (
        (addtocart_df['device_id'] == device) &
        (addtocart_df['product_id'] == product) &
        (addtocart_df['event_timestamp'] > ts) &
        (addtocart_df['event_timestamp'] <= ts + timedelta(minutes=5))
    ).any()

    # ② 30분 내 PURCHASE 존재 여부
    purchase_flag = (
        (purchase_df['device_id'] == device) &
        (purchase_df['product_id'] == product) &
        (purchase_df['event_timestamp'] > ts) &
        (purchase_df['event_timestamp'] <= ts + timedelta(minutes=30))
    ).any()

    result_rows.append({
        'device_id': device,
        'product_id': product,
        'addtocart_5min': int(addtocart_flag),
        'purchase_30min': int(purchase_flag)
    })

pageview_result = pd.DataFrame(result_rows)

# ---------------------------
# 🔹 product별 통계 계산
# ---------------------------

summary = (
    pageview_result
    .groupby('product_id')
    .agg(
        pageview_device_count=('device_id', 'nunique'),
        addtocart_5min_device_count=('addtocart_5min', 'sum'),
        purchase_30min_device_count=('purchase_30min', 'sum')
    )
    .reset_index()
)

# 비율 계산
summary['addtocart_5min_ratio'] = (summary['addtocart_5min_device_count'] / summary['pageview_device_count']).round(3)
summary['purchase_30min_ratio'] = (summary['purchase_30min_device_count'] / summary['pageview_device_count']).round(3)

summary = summary.sort_values('product_id').reset_index(drop=True)

# ---------------------------
# 결과 출력
# ---------------------------
summary.head(30)


,product_id,pageview_device_count,addtocart_5min_device_count,purchase_30min_device_count,addtocart_5min_ratio,purchase_30min_ratio
0,P001,10,1,2,0.100,0.200
1,P002,16,0,2,0.000,0.125
2,P003,17,2,1,0.118,0.059
3,P004,14,1,3,0.071,0.214
4,P005,12,2,1,0.167,0.083
5,P006,13,1,3,0.077,0.231
6,P007,7,1,3,0.143,0.429
7,P008,18,2,2,0.111,0.111
8,P009,14,1,1,0.071,0.071
9,P010,13,1,1,0.077,0.077
